In [ ]:
import nbdev.showdoc

In [ ]:
#default_exp export_sbt
#default_cls_lvl 3
from nbdev.showdoc import show_doc

In [ ]:
#export
from nbdev.imports import *
from fastcore.script import *
from fastcore.foundation import *
from keyword import iskeyword
import nbformat

from chisel_nbdev.export_scala import *

# Export to Scala SBT project 

> The functions that allow one to generate an SBT project from the generated .sc Scala scripts.

Each 

# Manage imports
> Ammonite style imports are not valid in SBT. It is required to convert these imports and support Scala packages.

Creates all of the folders required for SBT in the folder specified by `lib_path` in `settings.ini`.
- src
    - main
        - java
        - resource
        - scala
    - test
        - java
        - resource
        - scala

In [ ]:
#export
def init_sbt_layout():
    cfg = Config(cfg_name='settings.ini')
    path = cfg.path("lib_path")
    for d in ['main', 'test']:
        for dd in ['resource', 'scala', 'java']:
            src = path/'src'/d/dd
            src.mkdir(parents=True, exist_ok=True)

In [ ]:
init_sbt_layout()

Need to convert from
```$file.^.source.load_ivy```

to
```import package.module_name```

iff `module_name` is not part of the same `package` as this file

In [ ]:
#export
_re_ammonite_import = re.compile(r'^\s*import\s+\${1}.+', re.MULTILINE | re.VERBOSE)

In [ ]:
#export
def get_import_mod_names(import_line):
    try:
        l1, l2 = import_line.split(",")
        mod_name = l2.split(".")[0].strip()
    except ValueError:
        mod_name = import_line.split(".")[-1].strip()
    print(f"import statement: {import_line}, mod_name: {mod_name}")
    return mod_name

In [ ]:
l = ' import $file.^.source.load_ivy, load_ivy._  '
mod_name = get_import_mod_names(l)
test_eq(mod_name, 'load_ivy')    

import statement:  import $file.^.source.load_ivy, load_ivy._  , mod_name: load_ivy


In [ ]:
#export
def mod_in_same_package(mod_name, package):
    mod = get_nbdev_module()
    return f'{mod_name}.sc' in mod.packages.get(package)

In [ ]:
test_eq(mod_in_same_package("ModB", "ComposedExample"), True)

In [ ]:
#export
def get_mod_path(mod_name):
    mod_name = mod_name.strip(".sc") + ".sc"
    cfg = Config(cfg_name='settings.ini')
    path = cfg.path("lib_path")
    return path/mod_name

In [ ]:
#export
def get_mods_package(mod_name): 
    "Returns the package that this mod (.sc) is listed under"
    packages = get_nbdev_module().packages
    for p, ms in packages.items():
        for m in ms:
            if m == mod_name + '.sc': return p
    return None

In [ ]:
test_eq(get_mods_package('ModB'), 'ComposedExample')

In [ ]:
#export
def get_mods_code(mod_name):
    '''returns mod_name.sc contents as string (expects no suffix)'''
    mod_name = mod_name.strip(".sc")
    cfg = Config(cfg_name='settings.ini')
    path = cfg.path("lib_path")
    with open(path/f'{mod_name}.sc') as f: return f.read()

In [ ]:
#export
def _replace_amm_import(mod_name, mod_pack):
    #TODO more fine-grained imports
    return f'import {mod_pack}.{mod_name}._'

In [ ]:
#export 
def replace_amm_imports(code, package):
    code = code.split("\n")
    imps, outc = [], []
    for l in code:
        imp = _re_ammonite_import.match(l)
        if imp:
            mod_name = get_import_mod_names(imp.group(0))
            mod_pack = get_mods_package(mod_name)
            if mod_pack is None:
                print(f"Couldn't find {mod_name}, omitting {imp.group(0)}")
                continue
            skip_imp = mod_in_same_package(mod_name, package)
#             print(f'{mod_name} is in package {package} == {skip_imp}')
            if skip_imp: 
                #omit the import as they share a namespace
                continue
            else:
                imps.append(_replace_amm_import(mod_name, mod_pack))
                continue
        outc.append(l)
    return "\n".join(outc), "\n".join(imps)

In [ ]:
code = f' import $file.^.source.load_ivy, load_ivy._  \n some code \n import $file.^.lib_name.mod_name\n '
imps = replace_amm_imports(code, 'Examples')
# imps = [l for l in split_amm_imports(code).split("\n")]

import statement:  import $file.^.source.load_ivy, load_ivy._  , mod_name: load_ivy
Couldn't find load_ivy, omitting  import $file.^.source.load_ivy, load_ivy._  
import statement:  import $file.^.lib_name.mod_name, mod_name: mod_name
Couldn't find mod_name, omitting  import $file.^.lib_name.mod_name


In [ ]:
#export
def prepare_scala(code, mod_name):
    package = get_mods_package(mod_name)
    code, imports = replace_amm_imports(code, package)
    # insert the package statment (TODO add com.blah.package from settings.ini)
    imports = f'package {package}\n' + imports
    # wrap the script's code in an object
    wrapped = wrap_code(code, mod_name)
    # combine imports and code to form single string 
    return imports + "\n\n" + wrapped

In [ ]:
#export
def wrap_code(code, mod_name):
    """Wraps the code block in an object named `mod_name`"""
    tabbed_code = "\n".join([f'    {l}' for l in code.split("\n")])
    return f'object {mod_name}' + ' {\n' + f'{tabbed_code}' + '\n}'

In [ ]:
print(wrap_code("import blah \nval a = 4\n", 'modA'))

object modA {
    import blah 
    val a = 4
    
}


In [ ]:
# test_nb = read_nb('import_composed_mod.ipynb')
# cell = test_nb['cells'][3]
mod = 'ModB'
code = get_mods_code(mod)
# print(code)
scala = prepare_scala(code, mod)
# print(scala)

import statement: import $file.^.nbdev.ModA, ModA._, mod_name: ModA
import statement: import $file.^.nbdev.test, test._, mod_name: test
import statement: import $file.^.source.load_ivy, mod_name: load_ivy
Couldn't find load_ivy, omitting import $file.^.source.load_ivy


# script2scala()
> Replaces the Ammonite imports with SBT compatible imports and wraps code in `Object`.
mod_name = name of the .sc script

In [ ]:
#export
def script2scala(mod_name):
    "Converts a Scala script (.sc) to a Scala file (.scala)"
    script_code = get_mods_code(mod_name)
        
    # replace imports and wrap code in object
    scala_code = prepare_scala(script_code, mod_name)
    
    return scala_code

# create_packages
> Creates the .scala files from the .sc scripts and the package data contained in `_.nbdev.py`

In [ ]:
#export
def create_packages():
    "Create directory for each package and export respective scala scripts `files` under `modules`"
    mod = get_nbdev_module()
    cfg = Config(cfg_name='settings.ini')
    path = cfg.path("lib_path")/'src'/'main'/'scala'
    for package, modules in mod.packages.items():
        pname = path/package
        pname.mkdir(parents=True, exist_ok=True)
        for mod in modules:
            mod_path = pname/f'{mod + "ala"}'
#             file_path = cfg.path("lib_path")/mod
#             shutil.copy(file_path, mod_name)
            scala_code = script2scala(mod.strip(".sc"))
            with open(mod_path, 'w') as f:
                f.write(scala_code)
            print(f'copied {mod} -> {package}/{mod + "ala"}')

In [ ]:
create_packages()

import statement: import $file.^.nbdev.ModB, ModB._, mod_name: ModB
import statement: import $file.^.source.load_ivy, mod_name: load_ivy
Couldn't find load_ivy, omitting import $file.^.source.load_ivy
copied ModC.sc -> ComposedExample/ModC.scala
import statement: import $file.^.source.load_ivy, mod_name: load_ivy
Couldn't find load_ivy, omitting import $file.^.source.load_ivy
copied ModA.sc -> ComposedExample/ModA.scala
import statement: import $file.^.nbdev.ModA, ModA._, mod_name: ModA
import statement: import $file.^.nbdev.test, test._, mod_name: test
import statement: import $file.^.source.load_ivy, mod_name: load_ivy
Couldn't find load_ivy, omitting import $file.^.source.load_ivy
copied ModB.sc -> ComposedExample/ModB.scala
copied NewScript2.sc -> Examples/NewScript2.scala
import statement: import $file.^.source.load_ivy, mod_name: load_ivy
Couldn't find load_ivy, omitting import $file.^.source.load_ivy
copied test.sc -> Examples/test.scala
copied NewScript.sc -> Examples/NewScript

In [ ]:
#export
def write_build_dot_sbt():
    # don't re-write if exists
    pass